In [2]:
#!pip install xlrd
import pandas as pd
#import xlrd
from pymongo import MongoClient
import datetime
import pprint 

In [3]:
# client = MongoClient('localhost', 27017)
# db = client.project
# collection = db.google
# #3個collections 分別for正評 負評 中立評論
# negative_col = db.negative
# positive_col = db.positive
# neutral_col = db.neutral

In [96]:
#製作corpus.txt
# p_comment = positive_col.find({},{"c":1,"_id":0})
# n_comment = negative_col.find({},{"c":1,"_id":0})
# ne_comment = neutral_col.find({},{"c":1,"_id":0})
# corpus = open('corpus.txt', 'w', encoding='utf-8')
# for comment in p_comment : 
#     corpus.write(list(comment.values())[0]+"\n")
# for comment in n_comment : 
#     corpus.write(list(comment.values())[0]+"\n")
# for comment in ne_comment :     
#     corpus.write(list(comment.values())[0]+"\n")

In [119]:
import jieba
import operator
import jieba.analyse

In [120]:
#corpus_cut = []
#引入stopword字典（我把一些可能會影響情緒判斷的詞從stopword字典剔除了 ex 不 很 絕不 沒有 ）
stopwords_set = set()
with open("./stopword_cn.txt") as stopwords:
    for stopword in stopwords:
        stopwords_set.add(stopword.strip('\n'))

In [121]:
jieba.set_dictionary("./dict.txt.big")
jieba.load_userdict("./my_dict.txt")

Building prefix dict from /Users/lihsinhuei/project/text_mining/dict.txt.big ...
Loading model from cache /var/folders/y8/21z2wg7d1wd87xcw71ywj02h0000gn/T/jieba.u68039cad0712ea234ee73f49c4a12927.cache
Loading model cost 1.450 seconds.
Prefix dict has been built succesfully.


In [126]:
#將語料斷詞  去除停用詞
#corpus是與料集 read進來之後放到list裡面 
corpus = []
with open("corpus.txt") as f: 
    data = f.readlines()
    for comment in data:
        corpus.append(comment.replace("\n",""))
        
count = {}
output_cut_all = open('corpus_cut_all.txt', 'w', encoding='utf-8')
for line in corpus:
    words = jieba.cut(line,cut_all=True) #出來的是可迭代的generator
    for word in words:
        if word not in stopwords_set:
            output_cut_all.write(word + ' ')
            if word not in count:
                count[word] = 1
            else:
                count[word] = count[word] + 1
    output_cut_all.write('\n')

sorted_count= sorted(count.items(),key = operator.itemgetter(1),reverse=False) #把wordcount排序看看
sorted_count    

    

[('真確', 1),
 ('戰鬥有氧', 1),
 ('人數管控', 1),
 ('合騎', 1),
 ('NTD\n', 1),
 ('內學', 1),
 ('居中', 1),
 ('通暢', 1),
 ('暢通', 1),
 ('降', 1),
 ('綽綽有餘', 1),
 ('有餘', 1),
 ('小遊戲', 1),
 ('置放', 1),
 ('拋棄', 1),
 ('擦拭紙', 1),
 ('紙巾', 1),
 ('多少次', 1),
 ('連碰', 1),
 ('非要', 1),
 ('變溫', 1),
 ('運河', 1),
 ('河上', 1),
 ('社交', 1),
 ('超載', 1),
 ('傾力', 1),
 ('所述', 1),
 ('泳帽', 1),
 ('閉館', 1),
 ('服務水平', 1),
 ('能容', 1),
 ('滿有', 1),
 ('天井', 1),
 ('長方', 1),
 ('長方形', 1),
 ('方形', 1),
 ('全數', 1),
 ('優選', 1),
 ('Buffet', 1),
 ('滾輪', 1),
 ('殺價', 1),
 ('愛喝', 1),
 ('喝光', 1),
 ('帶好', 1),
 ('側', 1),
 ('抬腿', 1),
 ('精良', 1),
 ('負荷', 1),
 ('定制', 1),
 ('五晚', 1),
 ('日間', 1),
 ('熱烈', 1),
 ('汽機', 1),
 ('陰涼', 1),
 ('遮風', 1),
 ('遮風避雨', 1),
 ('比賽場', 1),
 ('比賽場地', 1),
 ('賽場', 1),
 ('南投縣', 1),
 ('縣境', 1),
 ('境內', 1),
 ('上數', 1),
 ('全裸', 1),
 ('白紙', 1),
 ('白紙黑字', 1),
 ('黑字', 1),
 ('亂砍', 1),
 ('亂收', 1),
 ('亂收費', 1),
 ('唯有', 1),
 ('裸體', 1),
 ('應能', 1),
 ('地平', 1),
 ('秋', 1),
 ('燈罩', 1),
 ('起球', 1),
 ('坐下', 1),
 ('邊線', 1),
 ('榨出', 1),
 ('多汗', 1),
 ('

In [182]:
#發現字彙量太多了  找出出現次數少於20的詞 之後去掉
rare_words = []
often_words = []
for key,value in count.items():
    if value < 20:
        rare_words.append(key.replace("\n",''))
    else:
        often_words.append(key.replace("\n",''))
        
        

In [180]:
len(rare_words)

13145

In [183]:
len(often_words)

1029

In [1]:
#再做一次語料斷詞 
#一樣是結巴全切割
#這次扣除掉出現次數少於20的詞（可以試試看剔除例如50次以下的）

corpus = []
with open("corpus.txt") as f:
    data = f.readlines()
    for comment in data:
        corpus.append(comment.replace("\n",""))

count_often = {}
corpus_often = open('corpus_often.txt', 'w', encoding='utf-8')
for line in corpus:
    words = jieba.cut(line,cut_all=True) #出來的是可迭代的generator
    print(words)
    for word in words:
        if word in rare_words:
            continue
        else:
            if word not in stopwords_set:
                corpus_often.write(word + ' ')
                if word not in count_often:
                    count_often[word] = 1
                else:
                    count_often[word] = count_often[word] + 1
    corpus_often.write('\n')

sorted_count_often= sorted(count_often.items(),key = operator.itemgetter(1),reverse=False)
sorted_count_often    

#結果有一些詞沒被剔除掉  要再檢查一下原因

FileNotFoundError: [Errno 2] No such file or directory: 'corpus.txt'

In [185]:
len(sorted_count_often)

1036

In [42]:
# !pip install sklearn
# !pip install gensim
from gensim.models import word2vec

### word2vec

In [186]:
sentences = word2vec.LineSentence("./corpus_often.txt")
model = word2vec.Word2Vec(sentences, size=50) #size是指定維度
model.save("word2vec.model")
#模型讀取方式
model = word2vec.Word2Vec.load("word2vec.model")
model.most_similar("教練")


/Users/lihsinhuei/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('專業', 0.9858165979385376),
 ('很', 0.9656957387924194),
 ('親切', 0.9582432508468628),
 ('好', 0.955270528793335),
 ('佳', 0.9415335655212402),
 ('熱心', 0.9353007078170776),
 ('服務態度', 0.9249941110610962),
 ('用心', 0.9225823879241943),
 ('認真', 0.9191426038742065),
 ('服務', 0.915899395942688)]

In [188]:
#模型讀取方式
model = word2vec.Word2Vec.load("word2vec.model")
model.most_similar("交通")
                          

/Users/lihsinhuei/.pyenv/versions/3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[('地點', 0.9663873910903931),
 ('方便', 0.9659215211868286),
 ('停車位', 0.9623551964759827),
 ('便利', 0.9584711194038391),
 ('車位', 0.9584277272224426),
 ('機車', 0.9540716409683228),
 ('少', 0.9530608057975769),
 ('大器', 0.9529093503952026),
 ('材種', 0.9478943943977356),
 ('太少', 0.94747394323349)]

### word2vec
原型：
class gensim.models.word2vec.Word2Vec(sentences=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000)

重要參數解說：
sentences:當然了，這是要訓練的句子集，沒有他就不用跑了
size:這表示的是訓練出的詞向量會有幾維
alpha:機器學習中的學習率，這東西會逐漸收斂到 min_alpha
sg:這個不是三言兩語能說完的，sg=1表示採用skip-gram,sg=0 表示採用cbow
window:還記得孔乙己的例子嗎？能往左往右看幾個字的意思
workers:執行緒數目，除非電腦不錯，不然建議別超過 4
min_count:若這個詞出現的次數小於min_count，那他就不會被視為訓練對象


### 詞頻

In [189]:
from  sklearn.feature_extraction.text  import  CountVectorizer  

In [190]:
#將文本中的詞語轉換為詞頻矩陣  
vectorizer = CountVectorizer()  
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [191]:
with open('corpus_often.txt') as f:
    file = f.readlines()
clean_file = []
for n in file:
    clean_file.append(n.replace("\n",""))
    

In [192]:
type(clean_file)

list

In [193]:
output = vectorizer.fit_transform(clean_file)
#output.toarray() #查看詞頻結果  
word = vectorizer.get_feature_names() #d看看我們的詞庫裡有什麼 
print(word)  

['cp', 'david', 'nt', 'trx', 'worldgym', '一下', '一中', '一些', '一位', '一個', '一個月', '一元', '一分', '一分鐘', '一半', '一堆', '一天', '一定', '一家', '一小', '一年', '一樓', '一次', '一流', '一直', '一起', '一間', '一點', '三個', '三思', '三溫暖', '上班', '上課', '上點', '下來', '下午', '下去', '下班', '下雨', '不一', '不便', '不到', '不可', '不多', '不夠', '不好', '不好停車', '不想', '不敢', '不斷', '不會', '不用', '不知', '不算', '不行', '不貴', '不足', '不錯', '世界', '中心', '主動', '主管', '主要', '之前', '之外', '之後', '乾淨', '事情', '二樓', '五星', '五臟', '五臟俱全', '交通', '享受', '人員', '人多', '人太多', '人少', '人很多', '人數', '人潮', '今天', '介紹', '仔細', '付費', '令人', '以上', '以使', '以來', '以前', '以後', '休息', '休閒', '伸展', '似乎', '但是', '但有', '位於', '位置', '佔用', '作人', '使用', '來說', '來體', '便利', '便宜', '保持', '保養', '俱全', '俱樂部', '值得', '假日', '偉大', '停車', '停車位', '停車場', '健康', '健身', '健身中心', '健身器', '健身器材', '健身工廠', '健身房', '健身運動', '備有', '備齊', '像是', '價位', '價格', '價格便宜', '價格合理', '價錢', '優惠', '優秀', '優良', '優質', '優點', '充滿', '充足', '免費', '兒童', '入口', '入場', '入會', '入會費', '內容', '內湖', '全家', '全新', '兩個', '公司', '公園', '公尺', '再來', '再次', '冷氣', '刁難', '分店', '分鐘', '划算', '前

In [194]:
from  sklearn.feature_extraction.text  import  TfidfTransformer  
transformer = TfidfTransformer()  
 
#將詞頻矩陣X統計成TF-IDF值  
tfidf = transformer.fit_transform(output)  
#查看數據結構 tfidf[i][j]表示i類文本中的tf-idf權重  
print(tfidf.toarray())


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [195]:
print(tfidf.toarray()[60]) #結果是稀疏矩陣  要再降維  然後把y加上 拿去跑回歸看看 

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [200]:
x = tfidf.toarray()

In [201]:
type(x)

numpy.ndarray